### Resource Creating Fuctions 

#### imports and credentials

In [ ]:
import logging
import boto3
from botocore.exceptions import ClientError
import json

ACCESS_KEY = '********************'
SECRET_KEY = '****************************************'

#### Lambda creation function 

In [ ]:

def create_lambda(lambda_name,location, S3_bucket, S3_key, key, value):
    """
    https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/lambda.html
    """
    
    lambda_ = boto3.client('lambda', region_name=location,
                           aws_access_key_id=ACCESS_KEY,
                           aws_secret_access_key=SECRET_KEY)
    try:

        response1 =  lambda_.create_function(FunctionName=lambda_name,
                                Runtime='python3.6',
                                Role='arn:aws:iam::453890640763:role/lambda-role',
                                Handler='function.lambda_handler',
                                Code={
                                    'S3Bucket':S3_bucket ,
                                    'S3Key': S3_key,
                                    }
                               )
        response2 = lambda_.tag_resource(
                                Resource=response1['FunctionArn'],
                                Tags={
                                    key : value
                                }
                            )
        
        logging.info(json.dumps(response1, indent=4))
        logging.info(json.dumps(response2, indent=4))
        
    except ClientError as e:
        logging.error(e)
        
    return response1
        

#### S3 Bucket creation function

In [ ]:
def create_bucket(bucket_name, location, key, value):
    """
    https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.create_bucket
    """
    s3 = boto3.client('s3',
                      aws_access_key_id=ACCESS_KEY,
                      aws_secret_access_key=SECRET_KEY            
                     )
    
    try:
        response1 = s3.create_bucket(ACL='private',
                                    Bucket=bucket_name,
                                    CreateBucketConfiguration={'LocationConstraint': location}
                                   )
        tag={'TagSet':[{'Key': key, 'Value': value}]}
        response2 = s3.put_bucket_tagging(Bucket=bucket_name, Tagging=tag)
        
        logging.info(json.dumps(response1, indent=4))
        logging.info(json.dumps(response2, indent=4))
    except ClientError as e:
        logging.error(e)
        print('error')
        
    return response1


#### Sqs creation function

In [ ]:
def create_sqs(queue_name, location):
    sqs = boto3.resource('sqs', region_name=location, 
                    aws_access_key_id=ACCESS_KEY,
                    aws_secret_access_key=SECRET_KEY)
    queue = sqs.create_queue(QueueName=queue_name, Attributes={'DelaySeconds': '0'}) #Attributes is optional 
    return queue

#### Sqs send message function

In [ ]:
def sqs_send_msg(queue_name, message):
    queue = sqs.get_queue_by_name(QueueName=queue_name)
    response = queue.send_message(MessageBody=message)
    return response

#### S3-lambda trigger creation function

In [ ]:
def create_s3_trigger(bucket_name, lambda_arn, location):
    
    #lambda1 and s3 are the client objects of lambda and s3_bucket respectively
    s3 = boto3.client('s3',
                    aws_access_key_id=ACCESS_KEY,
                    aws_secret_access_key=SECRET_KEY)
    
    lambda_ = boto3.client('lambda', region_name=location,
                       aws_access_key_id=ACCESS_KEY,
                       aws_secret_access_key=SECRET_KEY)
    
    response1 = lambda_.add_permission(FunctionName=lambda_arn,
                                       StatementId=bucket_name+'response1-id-1',
                                       Action='lambda:InvokeFunction',
                                       Principal='s3.amazonaws.com',
                                       SourceArn='arn:aws:s3:::'+bucket_name
                                      )
    response2 = lambda_.get_policy(FunctionName=lambda_arn)
    
    #s3 is s3 bucket boto3 client
    response3 = s3.put_bucket_notification_configuration(
                                    Bucket=bucket_name,
                                    NotificationConfiguration= {'LambdaFunctionConfigurations':[{'LambdaFunctionArn': lambda_arn, 'Events': ['s3:ObjectCreated:*']}]})


#### sqs-lambda trigger creation functionm

In [ ]:
def create_sqs_trigger(sqs_arn, lambda_arn, location):
    
    lambda_ = boto3.client('lambda', region_name=location,
                       aws_access_key_id=ACCESS_KEY,
                       aws_secret_access_key=SECRET_KEY)
    
    response1 = lambda_.create_event_source_mapping(
    EventSourceArn=sqs_arn, #this sqs arn can be replaced with kinesis or dynamoDB arn
    FunctionName=lambda_arn,
    Enabled=False
#     BatchSize=123
#     StartingPosition='TRIM_HORIZON'|'LATEST'|'AT_TIMESTAMP',
#     StartingPositionTimestamp=datetime(2015, 1, 1)
    )
    

location = 'ap-south-1'
# lambda_response = create_lambda()
# sqs_response = create_sqs()
# sqs_arn = sqs_response['FunctionArn']
# lambda_arn = lambda_response['FunctionArn']

# create_sqs_trigger(sqs_arn, lambda_arn, location)
sqs_arn = 'arn:aws:sqs:ap-south-1:453890640763:first-queue'
lambda_arn = 'arn:aws:lambda:ap-south-1:453890640763:function:sqs'
create_sqs_trigger(sqs_arn, lambda_arn, location)

### Master Lambda Creation

In [ ]:
lambda_name = 'master'
location = 'ap-south-1'
key = 'flag'
value = 'masterlambdaflag'
S3_bucket = 'sannzay2' #bucket name
S3_key = 'function.zip'

create_lambda(lambda_name,location, S3_bucket, S3_key, key, value)

#fuction.zip contains lambda_function.py(along with the required packages installed) which has the following code.
#Make sure the handler in lambda is set to lambda_function.lambda_handler


```
import json, boto3, time

ACCESS_KEY = '********************'
SECRET_KEY = '****************************************'

def lambda_handler(event, context):
    #print(event['queryStringParameters']['name'])	
    print(event['name'])
    
    bucketname = event['name'] + 's3'
    key = 'flag'
    value = event['name'] + '_flag'
    
    data = f'''{{
	"bucketname" : "{bucketname}",
	"key" : "{key}",
	"value" : "{value}"
}}'''
 
    with open('/tmp/temp.json', 'w') as f:
        f.write(data)
    with open('/tmp/temp.json', 'r') as r:
        file_object = r.read()


        client = boto3.client('s3',
                        aws_access_key_id=ACCESS_KEY,
                        aws_secret_access_key=SECRET_KEY)

        client.put_object(Body=file_object, Bucket='slave-1', Key='config.json')
        client.put_object(Body=file_object, Bucket='slave-2', Key='config.json')
        client.put_object(Body=file_object, Bucket='slave-3', Key='config.json')

```

### Slave s3 buckets creation

#### slave-s3-1

In [ ]:
bucket_name = 'slave-1'
location = 'ap-south-1'
key = 'flag'
value = 'slave-1-bucketflag'

create_bucket(bucket_name, location, key, value)

#### slave-s3-2

In [ ]:
bucket_name = 'slave-2'
location = 'ap-south-1'
key = 'flag'
value = 'slave-2-bucketflag'

create_bucket(bucket_name, location, key, value)

#### slave-s3-3

In [ ]:
bucket_name = 'slave-3'
location = 'ap-south-1'
key = 'flag'
value = 'slave-3-bucketflag'

create_bucket(bucket_name, location, key, value)

### Slave lambda creation 

#### Slave-lambda-1

In [ ]:
lambda_name = 'slave-1'
location = 'ap-south-1'
key = 'flag'
value = 'slave-1-lambdaflag'
S3_bucket = 'sannzay2'
S3_key = 'function.zip'

slave_1_lambda = create_lambda(lambda_name,location, S3_bucket, S3_key, key, value)

#fuction.zip contains lambda_function.py(along with the required packages installed) which has the following code.
#Make sure the handler in lambda is set to lambda_function.lambda_handler

```
import json, boto3, time

ACCESS_KEY = '********************'
SECRET_KEY = '****************************************'

def create_bucket(bucket_name, location, key, value):
    """
    https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.create_bucket
    """
    s3 = boto3.client('s3',
                      aws_access_key_id=ACCESS_KEY,
                      aws_secret_access_key=SECRET_KEY,
#                       region_name=location
                     
                     )
    

    response1 = s3.create_bucket(ACL='private',
                                    Bucket=bucket_name,
                                    CreateBucketConfiguration={'LocationConstraint': location}
                                   )
    tag={'TagSet':[{'Key': key, 'Value': value}]}
    response2 = s3.put_bucket_tagging(Bucket=bucket_name, Tagging=tag)
        
    #logging.info(json.dumps(response1, indent=4))
    #logging.info(json.dumps(response2, indent=4))

#return response1

def lambda_handler(event, context):
    print('Below is the event\n')
    print(event)
    s3 = boto3.client('s3',
                      aws_access_key_id=ACCESS_KEY,
                      aws_secret_access_key=SECRET_KEY,
#                       region_name=location
                     
                     )    
    s3.download_file('slave-1', 'config.json', '/tmp/config.json')

    with open('/tmp/config.json', 'r') as r:
        data = r.read()
    print(type(data))
    print(data)
    file_ = json.loads(data)
    print('Text is below\n')
    print(file_)
    print(file_['bucketname'])
    bucketname = file_['bucketname']
    
    
    location = 'ap-south-1'
    key = file_['key']
    value = file_['value']
    
    create_bucket(bucketname, location, key, value)
```

#### Slave-lambda-2

In [ ]:
lambda_name = 'slave-2'
location = 'ap-south-1'
key = 'flag'
value = 'slave-2-lambdaflag'
S3_bucket = 'sannzay2'
S3_key = 'function.zip'

slave_2_lambda = create_lambda(lambda_name,location, S3_bucket, S3_key, key, value)

#fuction.zip contains lambda_function.py(along with the required packages installed).
#Make sure the handler in lambda is set to lambda_function.lambda_handler

#### Slave-lambda-3

In [ ]:
lambda_name = 'slave-3'
location = 'ap-south-1'
key = 'flag'
value = 'slave-3-lambdaflag'
S3_bucket = 'sannzay2'
S3_key = 'function.zip'

slave_1_lambda = create_lambda(lambda_name,location, S3_bucket, S3_key, key, value)

#fuction.zip contains lambda_function.py(along with the required packages installed).
#Make sure the handler in lambda is set to lambda_function.lambda_handler

### Attaching all the triggers to respective resources

#### Attaching respective s3-slaves to lambda-slaves

In [ ]:
location = 'ap-south-1'
bucket_name = 'slave-1'

lambda_arn = slave_1_lambda['FunctionArn']

create_s3_trigger(bucket_name, lambda_arn, location)

In [ ]:
location = 'ap-south-1'
bucket_name = 'slave-2'

lambda_arn = slave_2_lambda['FunctionArn']

create_s3_trigger(bucket_name, lambda_arn, location)

In [ ]:
location = 'ap-south-1'
bucket_name = 'slave-2'

lambda_arn = slave_2_lambda['FunctionArn']

create_s3_trigger(bucket_name, lambda_arn, location)

In [ ]:
## Now attach the Master-lambda to the api-gateway and trigger with api

## When the api is created, create a 'GET' method and assign the lambda name there. In the 'Method execution' page,
#click the Integration request' section and go to the'Mapping templates' and type 'application/json' and 
# enter ["name" : "$input.params('name')"}'] in the json format in the space provided.

## The api name attached to the master-lambda is 'master_trigger'

## The url of that is "https://rm2tbrgb67.execute-api.ap-south-1.amazonaws.com/initial_stage"

## Attach the required message with the link as:
#"https://rm2tbrgb67.execute-api.ap-south-1.amazonaws.com/initial_stage?name=brandson" with message=brandson

In [5]:
## The master-lambda is triggered by an api-gateway with a message.

## When triggered this lambda function creates a file containing name,key,value generated with the help 
# of message and save it into the three slave buckets.

## Those buckets inturn triggers 3 individual lambda functions namely, slave-1, slave-2 and slave-3.

## slave1-lambda function is designed to create an s3 bucket with the 
# information present in the file which is used to trigger it.

## Likewise slave-2 and slave-3 lambdas can be designed to create required resources for the client.